In [ ]:
import pandas as pd
import numpy as np
import re
import glob
from pathlib import Path
from datetime import datetime

In [ ]:
ts = datetime.now().strftime('%Y%m%d')

## Cars24

In [ ]:
a = pd.concat(pd.read_csv(f) for f in glob.glob(f"output/csv/{ts}/cars24/raw_csv/*.csv"))
a.head(15)

In [ ]:
a.info()
a.describe()

In [ ]:
a = a.dropna()
a = a.drop_duplicates(subset=['link_product'])

In [ ]:
"""Normalize transmissions"""
a['transmissions'] = (a['transmissions'].str.replace(r'\d+\s','',regex=True))

""" change price to int"""
a['price'] = (a['price'].str.replace(r'[\$,]','',regex=True)).astype(int)

"""add column year made"""
a['year_made'] = a['name'].str[:4].astype(int)

""" add columm car brand"""
pattern = re.compile(r'^\d{4}\s+([a-zA-z]+)')
a['brand']=a['name'].str.extract(pattern, expand = False)

""" add column car name"""
pattern = re.compile(r'^\s*\d{4}\s+\S+\s+(.*)\s[MY]+\d{2}')
a ['name'] = a['name'].str.extract(pattern, expand = False)

""" change odometer to int"""
a['odometer'] = (a['odometer'].str.replace(r'[\s[kms]','',regex=True)).astype(int)
for idx, rows in a['odometer'].items():
    if rows > 0 and rows <= 20:
        a.loc[idx, 'odometer'] = '0 - 20,000'
    elif rows > 20 and rows <= 40:
        a.loc[idx, 'odometer'] = '20,001 - 40,000'
    elif rows > 40 and rows <= 60:
        a.loc[idx, 'odometer'] = '40,001 - 60,000'
    elif rows > 60 and rows <= 80:
        a.loc[idx, 'odometer'] = '60,001 - 80,000'
    elif rows > 80 and rows <= 100:
        a.loc[idx, 'odometer'] = '80,001 - 100,000'
    elif rows > 100 and rows <= 120:
        a.loc[idx, 'odometer'] = '100,001 - 120,000'
    elif rows > 120 and rows <= 140:
        a.loc[idx, 'odometer'] = '120,001 - 140,000'
    elif rows > 140 and rows <= 160:
        a.loc[idx, 'odometer'] = '140,001 - 160,000'
    elif rows > 160 and rows <= 180:
        a.loc[idx, 'odometer'] = '160,001 - 180,000'
    elif rows > 180 and rows <= 200:
        a.loc[idx, 'odometer'] = '180,001 - 200,000'
    else:
        a.loc[idx, 'odometer'] = '200,001+'

In [ ]:
a

In [ ]:
output = Path ('output') / 'csv' / f'{ts}' / 'cars24' / 'clean csv'
output.mkdir(parents=True, exist_ok=True)
filename= f'{ts}_cars24_clean.csv'
file_path = output / filename
a.to_csv(file_path, index=False, encoding="utf-8-sig")

## cheapcarco

In [ ]:
df = pd.concat(pd.read_csv(f) for f in glob.glob(f'output/csv/{ts}/cheapcarco/raw_csv/*.csv'))
df.head(15)

In [ ]:
df.info()

In [ ]:
df.drop_duplicates(subset=['link_product'])
df = df.dropna()
df

In [ ]:
"""Normalize transmissions"""
df['transmissions']= df['transmissions'].str.replace('Sports Automatic Dual Clutch','Automatic')
df['transmissions']= df['transmissions'].str.replace('Sports Automatic','Automatic')
df['transmissions']= df['transmissions'].str.replace('Constant Variable','Automatic')

"""Normalize fuel_type"""
df['fuel_type']= df['fuel_type'].str.replace('Petrol - Unleaded','Petrol')
df['fuel_type']= df['fuel_type'].str.replace('Petrol - Premium ULP','Petrol')

""" change price to int"""
df = df[df['price'] != 'P.O.A']
df['price'] = (df['price'].str.replace(r'[\$,]','',regex=True)).astype(int)

"""add column year made"""
df['year_made'] = df['name'].str[:4].astype(int)

""" add columm car brand"""
pattern = re.compile(r'^\d{4}\s+([a-zA-z]+)')
df['brand']=df['name'].str.extract(pattern, expand = False)

""" add column car name"""
pattern = re.compile(r'^\s*\d{4}\s+\S+\s+(.*)')
df ['name'] = df['name'].str.extract(pattern, expand = False)

""" change odometer to int"""
df['odometer'] = (df['odometer'].str.replace(r'[\s[Kkms,]','',regex=True)).astype(int)
for idx, rows in df['odometer'].items():
    if rows > 0 and rows <= 20000:
        df.loc[idx, 'odometer'] = '0 - 20,000'
    elif rows > 20000 and rows <= 40000:
        df.loc[idx, 'odometer'] = '20,001 - 40,000'
    elif rows > 40000 and rows <= 60000:
        df.loc[idx, 'odometer'] = '40,001 - 60,000'
    elif rows > 60000 and rows <= 80000:
        df.loc[idx, 'odometer'] = '60,001 - 80,000'
    elif rows > 80000 and rows <= 100000:
        df.loc[idx, 'odometer'] = '80,001 - 100,000'
    elif rows > 100000 and rows <= 120000:
        df.loc[idx, 'odometer'] = '100,001 - 120,000'
    elif rows > 120000 and rows <= 140000:
        df.loc[idx, 'odometer'] = '120,001 - 140,000'
    elif rows > 140000 and rows <= 160000:
        df.loc[idx, 'odometer'] = '140,001 - 160,000'
    elif rows > 160000 and rows <= 180000:
        df.loc[idx, 'odometer'] = '160,001 - 180,000'
    elif rows > 180000 and rows <= 200000:
        df.loc[idx, 'odometer'] = '180,001 - 200,000'
    else:
        df.loc[idx, 'odometer'] = '200,001+'

In [ ]:
df

In [ ]:
output = Path ('output') / 'csv' / f'{ts}' / 'cheapcarco' / 'clean csv'
output.mkdir(parents=True, exist_ok=True)
filename= f'{ts}_cheapcarco_clean.csv'
file_path = output / filename
df.to_csv(file_path, index=False, encoding="utf-8-sig")

## brisbaneautogroup

In [ ]:
df = pd.concat(pd.read_csv(f) for f in glob.glob(f'output/csv/{ts}/brisbaneautogroup/raw_csv/*.csv'))
df.head(15)

In [ ]:
df.drop_duplicates(subset=['link_product'])
df = df.dropna()

In [ ]:
df.info()

In [ ]:
"""Normalize transmissions"""
df['transmissions'] = (df['transmissions'].str.replace(r'\d+\s','',regex=True))
df['transmissions']= df['transmissions'].str.replace('SP Sports Automatic','Automatic')
df['transmissions']= df['transmissions'].str.replace('SP Automatic','Automatic')
df['transmissions']= df['transmissions'].str.replace('SP Constantly Variable','Automatic')
df['transmissions']= df['transmissions'].str.replace('SP Sports Automatic Dual Clutch','Automatic')
df['transmissions']= df['transmissions'].str.replace('SP Manual','Manual')
df['transmissions']= df['transmissions'].str.replace('Automatic Dual Clutch','Automatic')

""" change price to int"""
df['price'] = (df['price'].str.replace(r'[\$,]','',regex=True)).astype(int)

"""Normalize fuel_type"""
df['fuel_type']= df['fuel_type'].str.replace('Petrol - Unleaded ULP','Petrol')
df['fuel_type']= df['fuel_type'].str.replace('Petrol - Premium ULP','Petrol')
df['fuel_type']= df['fuel_type'].str.replace('Hybrid with Petrol - Unleaded ULP','Petrol')

""" change odometer to int"""
df['odometer'] = (df['odometer'].str.replace(r'[\s[Kkms,]','',regex=True)).astype(int)
for idx, rows in df['odometer'].items():
    if rows > 0 and rows <= 20000:
        df.loc[idx, 'odometer'] = '0 - 20,000'
    elif rows > 20000 and rows <= 40000:
        df.loc[idx, 'odometer'] = '20,001 - 40,000'
    elif rows > 40000 and rows <= 60000:
        df.loc[idx, 'odometer'] = '40,001 - 60,000'
    elif rows > 60000 and rows <= 80000:
        df.loc[idx, 'odometer'] = '60,001 - 80,000'
    elif rows > 80000 and rows <= 100000:
        df.loc[idx, 'odometer'] = '80,001 - 100,000'
    elif rows > 100000 and rows <= 120000:
        df.loc[idx, 'odometer'] = '100,001 - 120,000'
    elif rows > 120000 and rows <= 140000:
        df.loc[idx, 'odometer'] = '120,001 - 140,000'
    elif rows > 140000 and rows <= 160000:
        df.loc[idx, 'odometer'] = '140,001 - 160,000'
    elif rows > 160000 and rows <= 180000:
        df.loc[idx, 'odometer'] = '160,001 - 180,000'
    elif rows > 180000 and rows <= 200000:
        df.loc[idx, 'odometer'] = '180,001 - 200,000'
    else:
        df.loc[idx, 'odometer'] = '200,001+'

"""add column year made"""
df['year_made'] = df['name'].str[:4].astype(int)

""" add columm car brand"""
pattern = re.compile(r'^\d{4}\s+([a-zA-z]+)')
df['brand']=df['name'].str.extract(pattern, expand = False)

""" add column car name"""
pattern = re.compile(r'^\s*\d{4}\s+\S+\s+(.*)')
df ['name'] = df['name'].str.extract(pattern, expand = False)

In [ ]:
df

In [ ]:
output = Path ('output') / 'csv' / f'{ts}' / 'brisbaneautogroup' / 'clean csv'
output.mkdir(parents=True, exist_ok=True)
filename= f'{ts}_brisbaneautogroup_clean.csv'
file_path = output / filename
a.to_csv(file_path, index=False, encoding="utf-8-sig")